In [3]:
from __future__ import print_function
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler,VectorIndexer
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
if __name__ == "__main__":
    spark = SparkSession\
        .builder\
        .appName("randomForestRegression")\
        .getOrCreate()

In [5]:
dataset = spark.read.csv("Admission_Prediction.csv",header=True)

In [6]:
dataset.show()

+---------+-----------+-----------------+----+----+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating| SOP| LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+----+----+----+--------+---------------+
|   337.00|     118.00|                4|4.50|4.50|9.65|    1.00|           0.92|
|   324.00|     107.00|                4|4.00|4.50|8.87|    1.00|           0.76|
|     null|     104.00|                3|3.00|3.50|8.00|    1.00|           0.72|
|   322.00|     110.00|                3|3.50|2.50|8.67|    1.00|           0.80|
|   314.00|     103.00|                2|2.00|3.00|8.21|    0.00|           0.65|
|   330.00|     115.00|                5|4.50|3.00|9.34|    1.00|           0.90|
|   321.00|     109.00|             null|3.00|4.00|8.20|    1.00|           0.75|
|   308.00|     101.00|                2|3.00|4.00|7.90|    0.00|           0.68|
|   302.00|     102.00|                1|2.00|1.50|8.00|    0.00|           0.50|
|   323.00|     

In [7]:
dataset.printSchema()

root
 |-- GRE Score: string (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR: string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit: string (nullable = true)



In [8]:
from pyspark.sql.functions import col
new_data = dataset.select(*(col(c).cast("float").alias(c) for c in dataset.columns))

In [9]:
new_data.printSchema()

root
 |-- GRE Score: float (nullable = true)
 |-- TOEFL Score: float (nullable = true)
 |-- University Rating: float (nullable = true)
 |-- SOP: float (nullable = true)
 |-- LOR: float (nullable = true)
 |-- CGPA: float (nullable = true)
 |-- Research: float (nullable = true)
 |-- Chance of Admit: float (nullable = true)



In [10]:
from pyspark.sql.functions import col, count, isnan, when

In [11]:
#checking for null ir nan type values in our columns
new_data.select([count(when(col(c).isNull(), c)).alias(c) for c in new_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|       15|         10|               15|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [12]:
from pyspark.ml.feature import Imputer

In [13]:
imputer = Imputer(inputCols=["GRE Score", "TOEFL Score","University Rating"], 
                  outputCols=["GRE Score", "TOEFL Score","University Rating"])
model = imputer.fit(new_data)

imputed_data = model.transform(new_data)

In [14]:
#checking for null ir nan type values in our columns
imputed_data.select([count(when(col(c).isNull(), c)).alias(c) for c in imputed_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|        0|          0|                0|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [15]:
features = imputed_data.drop('Chance of Admit')

In [16]:
#let's assemble our features together using vectorAssembler
assembler = VectorAssembler(
    inputCols=features.columns,
    outputCol="features")

In [17]:
output = assembler.transform(imputed_data)

In [18]:
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(output)

In [19]:
featureIndexer = featureIndexer.transform(output)

In [20]:
new_indexed_data = featureIndexer.select("indexedFeatures", "Chance of Admit")

In [21]:
type(new_indexed_data)

pyspark.sql.dataframe.DataFrame

In [22]:
training, test = new_indexed_data.randomSplit([0.7, 0.3])

In [23]:
training.show()

+--------------------+---------------+
|     indexedFeatures|Chance of Admit|
+--------------------+---------------+
|[290.0,100.0,1.0,...|           0.47|
|[290.0,104.0,4.0,...|           0.45|
|[293.0,97.0,2.0,2...|           0.64|
|[294.0,93.0,1.0,1...|           0.46|
|[294.0,95.0,1.0,1...|           0.49|
|[295.0,93.0,1.0,2...|           0.46|
|[295.0,96.0,2.0,1...|           0.47|
|[295.0,99.0,2.0,2...|           0.57|
|[296.0,95.0,2.0,3...|           0.44|
|[296.0,99.0,2.0,3...|           0.47|
|[296.0,101.0,1.0,...|            0.6|
|[297.0,96.0,2.0,2...|           0.43|
|[297.0,96.0,2.0,2...|           0.34|
|[297.0,98.0,2.0,2...|           0.59|
|[297.0,99.0,4.0,3...|           0.54|
|[297.0,100.0,1.0,...|           0.52|
|[298.0,92.0,1.0,2...|           0.51|
|[298.0,97.0,3.121...|           0.45|
|[298.0,98.0,2.0,1...|           0.44|
|[298.0,98.0,2.0,4...|           0.34|
+--------------------+---------------+
only showing top 20 rows



In [32]:
random_forest_reg = RandomForestRegressor(featuresCol="indexedFeatures",labelCol="Chance of Admit" )

In [33]:
# Train model.  This also runs the indexer.
model = random_forest_reg.fit(training)

In [34]:
# Make predictions.
predictions = model.transform(test)

In [35]:
predictions.show()

+--------------------+---------------+-------------------+
|     indexedFeatures|Chance of Admit|         prediction|
+--------------------+---------------+-------------------+
|[290.0,104.0,4.0,...|           0.45| 0.5123362215891627|
|[294.0,93.0,1.0,1...|           0.46|0.46568134389739946|
|[294.0,95.0,1.0,1...|           0.49| 0.4622543835814856|
|[296.0,99.0,2.0,3...|           0.47|0.47349796252344706|
|[298.0,97.0,3.121...|           0.45|0.47116945894306655|
|[298.0,100.0,3.0,...|           0.58| 0.6109175541877605|
|[299.0,102.0,3.0,...|           0.56| 0.6430289932387557|
|[300.0,98.0,1.0,2...|           0.61| 0.5851480553088958|
|[300.0,100.0,3.0,...|           0.64| 0.6854133298605898|
|[300.0,100.0,3.12...|           0.62| 0.6705232021549257|
|[300.0,102.0,2.0,...|           0.56| 0.5439914610145969|
|[300.0,102.0,3.0,...|           0.63| 0.6500768235829801|
|[301.0,100.0,3.0,...|           0.67| 0.6040419916036989|
|[301.0,102.0,3.0,...|           0.68|  0.65341563568227

In [36]:
evaluator = RegressionEvaluator(
    labelCol="Chance of Admit", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print ("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.0592243


In [37]:
spark.stop()